In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import pystan, GPy, scipy

In [ ]:
def plotFunctionSamples(param, plotKwargs={}, fillKwargs={}, **kwargs):
    plotKwargs.update(kwargs)
    fillKwargs.update(kwargs)
    
    plt.plot(x, param.mean(0), **plotKwargs)
    plt.fill_between(x, param.mean(0)-1.98*param.std(0), param.mean(0)+1.98*param.std(0), alpha=.1, **fillKwargs)

In [ ]:
code = """
data {
  int<lower=1> N;
  vector[N] y;
  real x[N];
}
parameters {
  //constrain hyperparameters to be positive
  real<lower=0> length_scale;
  real<lower=0> alpha;
  real<lower=0> sigma;
  vector[N] f_eta;
}
transformed parameters {
  vector[N] f;
  {
    // define unneeded variables inside brackets 
    // to prevent storage by sampler
    matrix[N, N] L_cov;
    matrix[N, N] cov;
    cov = cov_exp_quad(x, alpha, length_scale);
    for (n in 1:N)
      cov[n, n] = cov[n, n] + 1e-12;
    L_cov = cholesky_decompose(cov);
    f = L_cov * f_eta;
  }
}
model {
  length_scale ~ gamma(2, 2);
  alpha ~ normal(0, 1);
  sigma ~ normal(0, 1);
  f_eta ~ normal(0, 1);
  y ~ normal(f, sigma);
}
"""

model = pystan.StanModel(model_code=code)

In [ ]:
n = 20
x = np.linspace(-1,1,n)
k = GPy.kern.RBF(1)

cov = np.tile(k.K(x[:,None]), (2,2))
cov[:n, :n] += .1 * np.eye(n)

s = scipy.stats.multivariate_normal.rvs(np.zeros(2*n), cov)
y, f = s[:n], s[n:]

plt.plot(x, f)
plt.scatter(x, y)

In [ ]:
cfg = {
    'N': n,
    'y': y,
    'x': x
}

samples = model.sampling(data=cfg, iter=2000)
samples

In [ ]:
samp = samples.extract()

In [ ]:
samp.keys()

In [ ]:
plt.plot(x, samp['f'].mean(0))
plt.fill_between(x, samp['f'].mean(0)-1.98*samp['f'].std(0), samp['f'].mean(0)+1.98*samp['f'].std(0), alpha=.1)

plt.plot(x, f, c='C1')

# Add derivative sampling 

## Allow derivate prior

In [ ]:
code = """
data {
  int<lower=1> N;
  vector[N] y;
  real x[N];
}
parameters {
  //constrain hyperparameters to be positive
  real<lower=0> length_scale;
  real<lower=0> alpha;
  real<lower=0> sigma;
  vector[2*N] eta;
}
transformed parameters {
  vector[N] f;
  vector[N] df;
  
  {
    matrix[2*N,2*N] L_cov;
    matrix[N, N] cov;
    vector[N] K_div_f;
      matrix[N, N] dK;
      matrix[N, N] ddK;
      matrix[N, N] v_pred;
      vector[N] df_pred_mu;
      matrix[N, N] cov_df_pred;
      matrix[N, N] nug_pred;
      real diff;
      real lsInv = 1./length_scale/length_scale;

      nug_pred = diag_matrix(rep_vector(1e-12,N));
    cov = cov_exp_quad(x, alpha, length_scale);
    for (n in 1:N)
      cov[n, n] = cov[n, n] + 1e-12;
    L_cov = cholesky_decompose(cov);
    f = L_cov[1:N,1:N] * f_eta[1:N];
    
    //derivative
    K_div_f = mdivide_left_tri_low(L_cov, f);
      K_div_f = mdivide_right_tri_low(K_div_f',L_cov)';

      // compute dK: cov(df, f), and ddK: cov(df, df)
      dK = cov_exp_quad(x, alpha, length_scale);
      ddK = cov_exp_quad(x, alpha, length_scale);
      for (i in 1:N){
        for (j in 1:N){
          diff = x[i] - x[j];

          dK[i,j] = dK[i,j] * (-lsInv * diff);
          ddK[i,j] = ddK[i,j] * (1.-lsInv*diff*diff) * lsInv;
        }
      }
      
      df_pred_mu = (dK * K_div_f);

      v_pred = mdivide_left_tri_low(L_cov, dK');
      cov_df_pred = ddK - v_pred' * v_pred;
      L_df = cholesky_decompose(cov_df_pred);
      
      df = L_df * df_eta;
  }
}
model {
  length_scale ~ gamma(2, 2);
  alpha ~ normal(0, 1);
  sigma ~ normal(0, 1);
  f_eta ~ normal(0, 1);
  df_eta ~ normal(0, 1);
  y ~ normal(f, sigma);
}
"""

model_derivative_prior = pystan.StanModel(model_code=code)

In [ ]:
cfg = {
    'N': n,
    'y': y,
    'x': x
}

samples = model_derivative_prior.sampling(data=cfg, iter=2000)
samples

In [ ]:
samp = samples.extract()

In [ ]:
samp.keys()

In [ ]:
plotFunctionSamples(samp['f'])
plt.plot(x, f, c='C1')

plt.twinx()
plotFunctionSamples(samp['df'], color='C1')